
## P7: Résolvez des problèmes en utilisant des algorithmes en Python #3


Dans le 2nd jeu d'essai de la mission, il y a 956 actions ; l'explosion combinatoire rend impossible le traitement car il existerait 6.090821257124996 e+287 combinaisons.

Il faut donc procéder autrement.
L'algorithme du sac à dos où l'on parcourt un arbre des cas possibles en décidant de prendre ou non une action peut être optimisé.



### 1. Préparation des données



In [1]:
import csv as csv
import re as re
import time



In [2]:
try:
    import big_o
except ModuleNotFoundError:
    print('cherche',ModuleNotFoundError)
    !pip install --upgrade pip
    !pip install big_o
    import big_o

In [3]:
# constant
FILE = "data/p7-20-shares.csv" 
FIELDNAMES = ['name', 'cost', 'profit_share', 'profit']


BUDGET = 50000

In [4]:
 def fn_timer(function):
        
#    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.perf_counter_ns()
        result = function(*args, **kwargs)
        t1 = time.perf_counter_ns()
        elapsed = (t1-t0)/1000000000
        print(f"Total time running {function.__name__}: {str(elapsed)}s seconds")
        return result
    return function_timer

In [5]:
# nettoyer les données des fichiers en entrée
def clean_char(texte: str) -> str:
    """ on ne conserve que les caractères lisibles 
    les lettres, chiffres, ponctuations décimales et signes
    les valeurs negatives sont acceptées, du point de vue profit.
    """
    texte_propre = re.sub(r"[^a-zA-Z0-9\-\.\,\+]", "", texte.replace(',','.'))
    return texte_propre


### 2. Amélioration des données

Sur python le travail avec float est plus couteux qu'en entier.
Une solution est alors de multiplier par 100 cout, budget et profit car cela ne change pas
le résultat.


In [6]:
""" lecture, nettoyage et chargement en dict.
    les non valeurs NaN sont rejetées.
"""
action_dict = {}
try:
    with open(FILE, "r", newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file, fieldnames=FIELDNAMES, 
                                    delimiter=';', doublequote=False)
        # skip the header
        next(csv_reader)
        for idx, line in enumerate(csv_reader):
            clean_data = True
            if line[FIELDNAMES[0]] != "":
                cle = clean_char(line[FIELDNAMES[0]])
            else:
                print(f" line {idx} had missing share name; dropped.")
                clean_data = False
            if line[FIELDNAMES[1]] != "":
                cout = 100 * int(clean_char(line[FIELDNAMES[1]]))
            else:
                print(f" line {idx} had missing cost data; dropped.")
                clean_data = False
            if line[FIELDNAMES[3]] != "":
                gain = int(100 * float(clean_char(line[FIELDNAMES[3]])))
            else:
                print(f" line {idx} had missing profit data; dropped.")
                clean_data = False
            if (gain < 0) or (cout < 0):
                print(f" line {idx} had negative value; accepted but pls check.")
            if (clean_data):
                action_dict[cle] = (cout, gain)
except FileNotFoundError:
    print(f" fichier non trouvé, Merci de vérifier son nom {file_name} : {FileNotFoundError}")            
except IOError:
    print(f" une erreur est survenue à l'écriture du fichier {file_name} : {IOError}")            


In [7]:
action_dict
                

{'Action-1': (2000, 100),
 'Action-2': (3000, 300),
 'Action-3': (5000, 750),
 'Action-4': (7000, 1400),
 'Action-5': (6000, 1019),
 'Action-6': (8000, 2000),
 'Action-7': (2200, 154),
 'Action-8': (2600, 286),
 'Action-9': (4800, 624),
 'Action-10': (3400, 918),
 'Action-11': (4200, 714),
 'Action-12': (11000, 990),
 'Action-13': (3800, 874),
 'Action-14': (1400, 14),
 'Action-15': (1800, 54),
 'Action-16': (800, 64),
 'Action-17': (400, 48),
 'Action-18': (1000, 140),
 'Action-19': (2400, 504),
 'Action-20': (11400, 2052)}

### 3. Résolution en force brute récurrente (pour mémoire):
si on considère l'ensemble des 'actions' du porte-feuille potentiel, il existe de multiples combinaisons d''action'. Parmi toutes les combinaisons dont le cout est inférieur au budget d'investissement, l'une d'entre elles est optimale car elle fournit le plus grand profit.


Maintenant quand on prend une 'action' au hasard,
soit on selectionne cette 'action' comme partie de la solution
soit on ne la selectionne pas.

Quand on selectionne une 'action', il faut exprimer la valeur et le cout du porte-feuille en fonction de sa valeur avant sélection (pour introduire de la récurrence avec n fonction de n-1).
valeur(pf(i)) = valeur(pf(i-1)) + valeur(action(i))
cout(pf(i)) = cout(pf(i-1)) + cout(action(i))
ou avec un cout exprimé en budget restant:
budget_restant(pf(i)) = budget_restant(pf(i-1)) - cout(action(i))

Quand on ne selectionne pas une action, la valeur et le poids du porte-feuille sont inchangés.

Cas d'arrêt de la fonction récurrente : si tout le budget est épuisé ou si toutes les actions ont été considérées.

Formalisons un peu mieux:

In [8]:
# recursively check all combinations

def knap_sack_brute(budget, actions, actions_porte_feuille=[]):
    # tant qu'il reste des actions non traitées
    if actions:
        # si l'action n'était pas retenue
        profit_sans, liste_sans = knap_sack_brute(budget, actions[1:], actions_porte_feuille)
        # 1ère action du porte-feuille
        val = actions[0]
        cout = val[1]
        # si son cout est inférieur au budget
        if cout <= budget:
            # action prise, budget diminuée de son cout, et sous ensemble 
            #   d'action restante et selection de l'action courante 
            profit_avec, liste_avec = knap_sack_brute(budget - cout, actions[1:], actions_porte_feuille + [val])
            # Choix de l'optimum :
            if profit_sans < profit_avec:
                return profit_avec, liste_avec

        return profit_sans, liste_sans
    else:
        return sum([i[2] for i in actions_porte_feuille]), actions_porte_feuille
# This code was inspired by Algomius

In [9]:
if False:
    liste_actions = [(cle,val[0],val[1]) for cle,val in action_dict.items()]
    valeur, action_pf = knap_sack_brute(BUDGET,liste_actions, [])
    print('Un profit de ',valeur, ' pour la selection d\'action ', action_pf)

### 4. Résolution en programmation dynamique

L'idée est de mémoriser les étapes déja calculées.

In [10]:
def knap_sack_optimise(budget, actions):
    # initialize matrix of size budget x share number
    # columns are budget increasing value whereas every line is a share
    matrice = [[0 for x in range(budget + 1)] for x in range(len(actions) + 1)]
    # for every share in book ; +1 is the zero line 
    for i in range(1, len(actions) + 1):
        # for every budget value up to max BUDGET
        for cout_courant in range(1, budget + 1):
            # if enough money to buy the current share
            if actions[i-1][1] <= cout_courant:
                # take the max btw 
                # current share profit + the previous best solution for remaining cost btw courrent_budget - share cost 
                # the previous Max value of the same budget column
                matrice[i][cout_courant] = max(actions[i-1][2] + matrice[i-1][cout_courant-actions[i-1][1]], matrice[i-1][cout_courant])
            else:
            # if not enough money, keep the previous solution
                matrice[i][cout_courant] = matrice[i-1][cout_courant]

    # track back the selected shares
    w = budget
    n = len(actions)
    actions_porte_feuille = []

    while w >= 0 and n >= 0:
        e = actions[n-1]
        if matrice[n][w] == matrice[n-1][w-e[1]] + e[2]:
            actions_porte_feuille.append(e)
            w -= e[1]

        n -= 1

    return matrice[-1][-1], actions_porte_feuille
# This code also was inspired by Algomius

In [11]:
if True:
    liste_actions = [(cle,val[0],val[1]) for cle,val in action_dict.items()]
    valeur, action_pf = knap_sack_optimise(BUDGET,liste_actions)
    print('Un profit de ',valeur, ' pour la selection d\'action ', action_pf)

Un profit de  9907  pour la selection d'action  [('Action-20', 11400, 2052), ('Action-19', 2400, 504), ('Action-18', 1000, 140), ('Action-13', 3800, 874), ('Action-11', 4200, 714), ('Action-10', 3400, 918), ('Action-8', 2600, 286), ('Action-6', 8000, 2000), ('Action-5', 6000, 1019), ('Action-4', 7000, 1400)]


In [12]:
@fn_timer
def measure_optimise(budget, liste):
    return knap_sack_optimise(budget,liste)
@fn_timer
def measure_brute(budget, liste):
    return knap_sack_brute(budget,liste,[])

In [13]:
liste_actions = [(cle,val[0],val[1]) for cle,val in action_dict.items()]
measure_optimise(BUDGET,liste_actions)


Total time running measure_optimise: 1.0530114s seconds


(9907,
 [('Action-20', 11400, 2052),
  ('Action-19', 2400, 504),
  ('Action-18', 1000, 140),
  ('Action-13', 3800, 874),
  ('Action-11', 4200, 714),
  ('Action-10', 3400, 918),
  ('Action-8', 2600, 286),
  ('Action-6', 8000, 2000),
  ('Action-5', 6000, 1019),
  ('Action-4', 7000, 1400)])

In [14]:
liste_actions = [(cle,val[0],val[1]) for cle,val in action_dict.items()]
measure_brute(BUDGET,liste_actions)

Total time running measure_brute: 2.1838188s seconds


(9907,
 [('Action-4', 7000, 1400),
  ('Action-5', 6000, 1019),
  ('Action-6', 8000, 2000),
  ('Action-8', 2600, 286),
  ('Action-10', 3400, 918),
  ('Action-11', 4200, 714),
  ('Action-13', 3800, 874),
  ('Action-18', 1000, 140),
  ('Action-19', 2400, 504),
  ('Action-20', 11400, 2052)])